In [1]:
# DATA CLEANING PIPELINE
from google.colab import drive
import os, glob
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Set paths
BASE_PATH = "/content/drive/MyDrive/NGO Project/Raw data"
SBA_PATH = f"{BASE_PATH}/sba"
PROCESSED_PATH = f"{BASE_PATH}/processed"

# Create processed folder
os.makedirs(PROCESSED_PATH, exist_ok=True)

# Helper functions
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(" ", "_")
                  .str.replace("/", "_")
    )
    return df

def save_parquet(df, filename):
    path = f"{PROCESSED_PATH}/{filename}.parquet"
    df.to_parquet(path, index=False)
    print(f"✅ Saved: {path} ({os.path.getsize(path) / (1024 * 1024):.2f} MB)")

def load_any_file(path):
    if not os.path.exists(path):
        return None
    ext = os.path.splitext(path)[1].lower()
    if ext == ".csv":
        return pd.read_csv(path, encoding='latin-1')
    elif ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    else:
        print(f"⚠ Unsupported format: {path}")
        return None

# SBA DATA CLEANING
print("Processing SBA datasets...")

# Public up to 150k
csv_files = sorted(glob.glob(f"{SBA_PATH}/public_up_to_150k_*.csv"))
cols_to_keep = [
    "LoanNumber", "DateApproved", "BorrowerState", "LoanStatus",
    "InitialApprovalAmount", "CurrentApprovalAmount", "JobsReported",
    "NAICSCode", "BusinessType"
]

df_list = []
for file in csv_files:
    print(f"Loading {file}...")
    chunk = pd.read_csv(file, usecols=lambda c: c in cols_to_keep, encoding='latin-1')
    chunk = clean_column_names(chunk)
    df_list.append(chunk)

if df_list:
    public_up_to_150k = pd.concat(df_list, ignore_index=True)
    save_parquet(public_up_to_150k, "sba_public_up_to_150k")
    del public_up_to_150k, df_list
else:
    print("No SBA public_up_to_150k files found.")

# Public 150k plus
file_150k_plus = f"{SBA_PATH}/public_150k_plus_240930.csv"
if os.path.exists(file_150k_plus):
    df_150k_plus = pd.read_csv(file_150k_plus, usecols=lambda c: c in cols_to_keep, encoding='latin-1')
    df_150k_plus = clean_column_names(df_150k_plus)
    save_parquet(df_150k_plus, "sba_public_150k_plus")
    del df_150k_plus
else:
    print(f"File not found: {file_150k_plus}")

# FOIA 7a
foia_7a_file = f"{SBA_PATH}/foia-7a-fy2020-present-asof-250331.csv"
if os.path.exists(foia_7a_file):
    df_foia_7a = pd.read_csv(foia_7a_file, encoding='latin-1')
    df_foia_7a = clean_column_names(df_foia_7a)
    save_parquet(df_foia_7a, "sba_foia_7a")
    del df_foia_7a
else:
    print(f"File not found: {foia_7a_file}")

# bkstudy
bkstudy_files = sorted(glob.glob(f"{SBA_PATH}/bkstudy_*.csv"))
for file in bkstudy_files:
    df_bk = pd.read_csv(file, encoding='latin-1')
    df_bk = clean_column_names(df_bk)
    base_name = os.path.basename(file).replace(".csv", "")
    save_parquet(df_bk, f"sba_{base_name}")
    del df_bk

print("✅ SBA datasets processed.")

# CBECS DATA CLEANING
print("Processing CBECS datasets...")

cbecs_candidates = [f for f in os.listdir(BASE_PATH) if 'cbecs2018_final_public' in f.lower()]
if cbecs_candidates:
    cbecs_file = os.path.join(BASE_PATH, cbecs_candidates[0])
    print(f"Found: {cbecs_file}")
    df_cbecs = load_any_file(cbecs_file)
    if df_cbecs is not None:
        df_cbecs = clean_column_names(df_cbecs)
        save_parquet(df_cbecs, "cbecs2018_final_public")
        del df_cbecs
else:
    print("No CBECS dataset found.")

# OpenEI DATA CLEANING
print("Processing OpenEI datasets...")

openei_candidates = [f for f in os.listdir(BASE_PATH) if 'usurdb' in f.lower()]
if openei_candidates:
    openei_file = os.path.join(BASE_PATH, openei_candidates[0])
    print(f"Found: {openei_file}")
    df_openei = load_any_file(openei_file)
    if df_openei is not None:
        df_openei = clean_column_names(df_openei)
        save_parquet(df_openei, "openei_usurdb")
        del df_openei
else:
    print("No OpenEI dataset found.")

# Final Check
print("\n📂 Processed files ready for analysis:")
for file in sorted(glob.glob(f"{PROCESSED_PATH}/*.parquet")):
    size_mb = os.path.getsize(file) / (1024 * 1024)
    print(f"{os.path.basename(file)} - {size_mb:.2f} MB")

Mounted at /content/drive
Processing SBA datasets...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_10_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_11_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_12_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_1_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_2_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_3_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_4_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_5_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_6_240930.csv...
Loading /content/drive/MyDrive/NGO Project/Raw data/sba/public_up_to_150k_7_240930.csv...
Loading /content/drive/MyDrive/NGO Project/R

/tmp/ipython-input-3234115827.py:82: DtypeWarning: Columns (19,36,37,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df_foia_7a = pd.read_csv(foia_7a_file, encoding='latin-1')


✅ Saved: /content/drive/MyDrive/NGO Project/Raw data/processed/sba_foia_7a.parquet (19.83 MB)
✅ SBA datasets processed.
Processing CBECS datasets...
Found: /content/drive/MyDrive/NGO Project/Raw data/cbecs2018_final_public.csv
✅ Saved: /content/drive/MyDrive/NGO Project/Raw data/processed/cbecs2018_final_public.parquet (4.82 MB)
Processing OpenEI datasets...
Found: /content/drive/MyDrive/NGO Project/Raw data/usurdb.csv


/tmp/ipython-input-3234115827.py:37: DtypeWarning: Columns (3,10,29,363,393,398,403,408,413,418,438,443,448,453,458,473,478,483,493,498,503,508,518,523,528,533,543,548,553,558,563,568,573,578,583,588,593,598,603,608,613,618,623,628,633,638,643,648,653,658,663,668,673,678,683,688,693,698,703,708,711,724) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path, encoding='latin-1')


✅ Saved: /content/drive/MyDrive/NGO Project/Raw data/processed/openei_usurdb.parquet (10.67 MB)

📂 Processed files ready for analysis:
cbecs2018_final_public.parquet - 4.82 MB
openei_usurdb.parquet - 10.67 MB
sba_foia_7a.parquet - 19.83 MB
sba_public_150k_plus.parquet - 17.53 MB
sba_public_up_to_150k.parquet - 140.34 MB


In [3]:
# Tableau file creation
import pandas as pd
import pyarrow.dataset as ds # Import pyarrow dataset
import numpy as np # Import numpy

processed_path = "/content/drive/MyDrive/NGO Project/Raw data/processed"

# Load SBA data using pyarrow dataset for chunking
sba_file = f"{processed_path}/sba_public_up_to_150k.parquet"
use_cols = ["borrowerstate", "businesstype"]

print("Aggregating SBA data in chunks using pyarrow.dataset...")

# Create a PyArrow dataset
dataset = ds.dataset(sba_file, format="parquet")

agg_region = {}
agg_sector = {}

# Iterate through batches instead of chunks
batch_size = 100_000
for batch in dataset.to_batches(columns=use_cols, batch_size=batch_size):
    chunk = batch.to_pandas()
    # Adoption likelihood by state
    state_adoption = {
        "CA": 0.45, "NY": 0.40, "MA": 0.42,
        "TX": 0.25, "FL": 0.28, "IL": 0.30,
        "WV": 0.10, "WY": 0.12, "KY": 0.15
    }

    # Adoption likelihood by sector
    sector_adoption = {
        "Professional, Scientific, and Technical Services": 0.35,
        "Educational Services": 0.40,
        "Healthcare and Social Assistance": 0.32,
        "Manufacturing": 0.22,
        "Retail Trade": 0.18,
        "Construction": 0.15,
        "Transportation and Warehousing": 0.12,
    }

    def assign_adoption(row):
        state_rate = state_adoption.get(row["borrowerstate"], 0.20)
        sector_rate = sector_adoption.get(row["businesstype"], 0.20)
        combined_rate = (state_rate + sector_rate) / 2
        return np.random.rand() < combined_rate

    chunk["renewable_adopted"] = chunk.apply(assign_adoption, axis=1).astype(int)

    # Aggregate by region/state
    for state, grp in chunk.groupby("borrowerstate"):
        agg_region.setdefault(state, []).extend(grp["renewable_adopted"])

    # Aggregate by business type
    for sector, grp in chunk.groupby("businesstype"):
        agg_sector.setdefault(sector, []).extend(grp["renewable_adopted"])

# Convert aggregated lists to averages
adoption_by_region = pd.DataFrame([
    {"borrowerstate": k, "adoption_rate": sum(v) / len(v)}
    for k, v in agg_region.items()
])

adoption_by_sector = pd.DataFrame([
    {"businesstype": k, "adoption_rate": sum(v) / len(v)}
    for k, v in agg_sector.items()
])

# Save for Tableau
adoption_by_region.to_csv(f"{processed_path}/adoption_by_region.csv", index=False)
adoption_by_sector.to_csv(f"{processed_path}/adoption_by_sector.csv", index=False)

print("✅ Exported:")
print("- adoption_by_region.csv")
print("- adoption_by_sector.csv")

# Save merged_for_tableau with only aggregated data
# Note: This merge is done on the aggregated data, not the full dataset
# It's a cross merge here
merged_for_tableau = adoption_by_region.merge(adoption_by_sector, how="cross")
merged_for_tableau.to_csv(f"{processed_path}/merged_for_tableau.csv", index=False)

print("✅ Merged Tableau file saved.")

Aggregating SBA data in chunks using pyarrow.dataset...
✅ Exported:
- adoption_by_region.csv
- adoption_by_sector.csv
✅ Merged Tableau file saved.
